In [46]:
# Load Libraries & load environment variables

In [47]:
import os
from dotenv import load_dotenv
from data.driver import Neo4jDriver
import pandas as pd

load_dotenv()

True

In [48]:
# Set connection object
conn = Neo4jDriver(
    os.environ.get("NEO4J_URI"),
    os.environ.get("NEO4J_USERNAME"),
    os.environ.get("NEO4J_PASSWORD"),
)

db = os.environ.get("DB")

In [49]:
# Define query function
def query_data(query, conn=None, db=None):
    # Function to handle the updating the Neo4j database in batch mode.
    results = None
    results = conn.query(query, db=db)
    return results

In [50]:
# 1. Top 5 customers in terms of card purchases (max. amount)
query_1 = """MATCH (cust :Customer)-[:HAS_CARD]->(card :CardNumber)-[p :PURCHASED_AT]->(m :Merchant)
RETURN DISTINCT cust.firstName+" "+cust.lastName AS customerName,sum(p.amount) AS totalPurchaseAmount
ORDER BY totalPurchaseAmount desc
LIMIT 5
;"""

result_1 = query_data(query_1, conn, db)
resultDF1 = pd.DataFrame([c.data() for c in result_1])
print(resultDF1)

    customerName  totalPurchaseAmount
0     Greta Swan         1.978438e+06
1   Erick Ingram         1.935143e+06
2  Dani Flanders         1.882640e+06
3   Candace Shea         1.815146e+06
4    Matt Carter         1.802131e+06


In [51]:
# 2. Top 5 merchant in terms of card purchases (max. amount)
query_2 = """MATCH (card :CardNumber)-[p :PURCHASED_AT]->(m :Merchant)
RETURN DISTINCT m.merchantName, sum(p.amount) AS totalPurchaseAmount
ORDER BY totalPurchaseAmount desc
LIMIT 5
;"""

result_2 = query_data(query_2, conn, db)
resultDF2 = pd.DataFrame([c.data() for c in result_2])
print(resultDF2)

     m.merchantName  totalPurchaseAmount
0    It Smart Group         3.778415e+06
1       Areon Impex         3.617039e+06
2  21st Century Fox         3.591051e+06
3          Vodafone         3.566855e+06
4          Facebook         3.552316e+06


In [52]:
# 3. Customers with most account transfers
query_3 = """MATCH (acct1 :AccountNumber)-[t :TRANSFERRED_TO]->(acct2 :AccountNumber)
WITH acct1, count(t) AS transferCount
RETURN acct1, transferCount
ORDER BY transferCount
;"""

result_3 = query_data(query_3, conn, db)
resultDF3 = pd.DataFrame([c.data() for c in result_3])
print(resultDF3)

                               acct1  transferCount
0   {'accountNumber': '423-13-4375'}             10
1   {'accountNumber': '883-66-3870'}             10
2   {'accountNumber': '062-18-1716'}             10
3   {'accountNumber': '532-16-1750'}             10
4   {'accountNumber': '624-06-2107'}             10
..                               ...            ...
95  {'accountNumber': '281-25-1765'}             10
96  {'accountNumber': '625-82-2217'}             10
97  {'accountNumber': '312-78-2310'}             10
98  {'accountNumber': '430-04-5447'}             10
99  {'accountNumber': '650-63-6154'}             10

[100 rows x 2 columns]


In [53]:
# 4. Top merchant in terms of card purchases for an age band - 20-30yrs
query_4 = """MATCH (cust :Customer)-[:HAS_CARD]->(:CardNumber)-[p :PURCHASED_AT]->(m :Merchant)
WHERE toInteger(cust.age) >=20 AND cust.age <= 30
RETURN DISTINCT m.merchantName AS merchant, sum(p.amount) AS totalPurchasedAmount
ORDER BY totalPurchasedAmount
LIMIT 1
;"""

result_4 = query_data(query_4, conn, db)
resultDF4 = pd.DataFrame([c.data() for c in result_4])
print(resultDF4)

  merchant  totalPurchasedAmount
0   Danone         493206.400403


In [54]:
# 5. Identify purchases within one hour of transfer to that account.
query_5 = """MATCH (acct1 :AccountNumber)-[trans :TRANSFERRED_TO]->(acct2 :AccountNumber)<-[:HAS_ACCOUNT]-(cust :Customer)-[:HAS_CARD]->(card :CardNumber)-[purch :PURCHASED_AT]->(m :Merchant)
WITH trans.transactionDateTime AS transferDateTime, purch.transactionDateTime AS purchaseDateTime, purch.amount AS purchaseAmount, purch.transactionId AS purchaseTransaction, m.merchantName AS merchantName
WHERE transferDateTime < purchaseDateTime < transferDateTime + duration({hours: 1})
RETURN purchaseTransaction, purchaseAmount, merchantName, "Verify Transaction" AS statusFlag
;"""

result_5 = query_data(query_5, conn, db)
resultDF5 = pd.DataFrame([c.data() for c in result_5])
print(resultDF5)

    purchaseTransaction  purchaseAmount           merchantName  \
0                969872    18993.858570           Biolife Grup   
1                691099    17553.718970               Facebook   
2                397884     4070.203048               Facebook   
3                472792    18277.032860                 Comodo   
4                472943     1470.317650                 CarMax   
5                954599    10534.211440                 CarMax   
6                157815    17412.549910             Amazon.com   
7                183546    15503.043760             Apple Inc.   
8                162514     8592.970528             Apple Inc.   
9                272302    12425.406300               Vodafone   
10               709831    17622.203290                   Mars   
11               422499    18616.467680         It Smart Group   
12               771072     4111.829705         It Smart Group   
13               828102     4336.294040         It Smart Group   
14        